In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from tqdm import notebook

from scipy.spatial.distance import pdist, squareform

import re

from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from nltk.corpus import stopwords as nltk_stopwords

!pip install faiss-cpu
import faiss

import spacy


from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

In [ ]:
def load(path):
  df_1 = pd.read_csv(path + 'marketing_dealer.csv', sep=';')
  df_2 = pd.read_csv(path + 'marketing_dealerprice.csv', sep=';')
  df_3 = pd.read_csv(path + 'marketing_product.csv', sep=';')
  df_4 = pd.read_csv(path + 'marketing_productdealerkey.csv', sep=';')
  return df_1, df_2, df_3, df_4

In [ ]:
dealer, dealer_price, product, product_match = load('drive/MyDrive/Colab Notebooks/Procept/')

In [ ]:
def text_worker(name): #отдельно функция для предварительной очистки текста
    #добавляем пробел на явных стыках и переходах
    pattern = [r"([а-я])([a-zA-ZА-Я])", r"([А-Я])([A-Za-z])", r"([a-z])([A-Zа-яА-Я])"]
    for p in pattern:
        try:
            name = re.sub(p, "\\1 \\2", name)
        except:
            pass
    name = name.lower()
    bad = ['просепт','prosept50','prosept50,', 'prosepteco50','prosepteco50,','ultra', 'crystal', '-ая', 'prosept']
    good = ['prosept',' prosept50 ',' prosept50 ', ' prosepteco50 ', ' prosepteco50 ', ' ultra ', ' crystal ','','']
    for o,n in zip(bad, good):
        name = name.replace(o,n)
    return name

def change_equal(name):
    name = text_worker(name)
    dictionary={}
    try:
        article = re.search(r' \d+-\d+/?\d?[а-я]?',name)[0].strip()
        #print(article)
        name = name.replace(article,'')
    except:
        article = np.nan
    try:
        v = re.findall(r'\d+(?:[\.,]\d+)? ?(?:мл|кг|г|л|шт)',name)[-1]
        dimension = re.search(r'[млкгшт]+',v)[0]
        quantity = float(v.replace(dimension,'').replace(',','.'))
        if dimension == 'мл':
            quantity = quantity/1000
            dimension = 'л'
        elif dimension == 'г':
            quantity = quantity/1000
            dimension = 'кг'
        name = name.replace(v,'')
    except:
        dimension = np.nan
        quantity = 0
    finally:
        dictionary['name_new'] = ' '.join(re.sub(r'\W+', ' ', name).split(r'\W+'))
        dictionary['article'] = article
        dictionary['dimension'] = dimension
        dictionary['quantity'] = quantity
        return dictionary

In [ ]:
!python -m spacy download ru_core_news_sm
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])

2023-11-28 17:14:25.782345: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 17:14:25.782417: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 17:14:25.782472: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 17:14:27.225084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 37.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2023-11-28 17:14:43.715426: E tensorflow/compiler/xla/s

In [ ]:
def spacy_1(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    doc_ru = nlp_ru(text)
    return " ".join([token.lemma_ for token in doc_ru])

## Список дилеров

In [ ]:
# Основная информация
dealer.head()

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora


## Результаты работы парсера площадок дилеров (что сравниваем)

In [ ]:
parser_new = dealer_price['product_name'].apply(lambda x: change_equal(x)).apply(pd.Series)
try:
    dealer_price = dealer_price.drop(['name_new', 'article', 'dimension', 'quantity'],axis=1)
except:
    pass
dealer_price = pd.concat([dealer_price, parser_new],axis=1)

## Товары, производимые заказчиком (с чем сравниваем)

In [ ]:
#заполняем пропуски
name_columns = ['name_1c', 'ozon_name', 'name', 'wb_name']
product['name_1c'] = product['name_1c'].fillna(product['name'])
product['ozon_name'] = product['ozon_name'].fillna(product['name_1c'])
product[name_columns] = product[name_columns].fillna('')

name_1c = product['name_1c'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_1c_new'})['name_1c_new']
ozon_name = product['ozon_name'].apply(change_equal).apply(pd.Series).drop(['article'],axis=1).rename(columns={'name_new': 'ozon_name_new'})
name_1 = product['name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_new'})['name_new']
wb_name = product['wb_name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'wb_name_new'})['wb_name_new']
products1 = pd.concat([product, ozon_name, name_1c, wb_name, name_1],axis=1)

In [ ]:
products1['full_name'] = products1.apply(lambda x: ' '.join(list(set((x['name_1c_new']+' '+x['ozon_name_new']+' '+x['name_new']+' '+x['wb_name_new']).split()))), axis=1)

In [ ]:
#Финальная таблица без лишнего
products_final = products1.loc[:,['id','article','cost','recommended_price', 'full_name', 'dimension', 'quantity']]

In [ ]:
products_final['recommended_price'] = products_final['recommended_price'].fillna(products_final['recommended_price'].median())

## Матчинг товаров

In [ ]:
product_match.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


## Решения

In [ ]:
# Лемматизация
products_final['full_name'] = products_final['full_name'].apply(spacy_1)
dealer_price['name_new'] = dealer_price['name_new'].apply(spacy_1)

In [ ]:
# OHE
encoder = OneHotEncoder()

new_demensions = encoder.fit_transform(products_final[['dimension']]).toarray()
products_final = pd.concat([products_final, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)

new_demensions = encoder.transform(dealer_price[['dimension']]).toarray()
dealer_price_final = pd.concat([dealer_price, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)

In [ ]:
dealer_price_final.head()

,id,product_key,price,product_url,product_name,date,dealer_id,name_new,article,quantity,кг,л,шт,NaN
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,средство универсальный universal spray,NaN,0.5,0.0,1.0,0.0,0.0
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2,концентрат multipower для мытьё пол цитрус,NaN,1.0,0.0,1.0,0.0,0.0
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2,средство для чистка люстр universal anti dust,NaN,0.5,0.0,1.0,0.0,0.0
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2,удалитель ржавчина rust remover,023-05,0.5,0.0,1.0,0.0,0.0
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2,средство мыть для баня и сауна multipower wood,NaN,1.0,0.0,1.0,0.0,0.0


In [ ]:
dealer_price_final = dealer_price_final[['product_key', 'price', 'name_new', 'article', 'quantity', 'кг', 'л', 'шт', np.nan]]

In [ ]:
dealer_price_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_key  20416 non-null  object 
 1   price        20416 non-null  float64
 2   name_new     20416 non-null  object 
 3   article      4684 non-null   object 
 4   quantity     20416 non-null  float64
 5   кг           20416 non-null  float64
 6   л            20416 non-null  float64
 7   шт           20416 non-null  float64
 8   nan          20416 non-null  float64
dtypes: float64(6), object(3)
memory usage: 1.4+ MB


In [ ]:
base = products_final.drop(['cost'], axis=1).copy()
base = base.set_index('id')

In [ ]:
base.head()

,article,recommended_price,full_name,quantity,кг,л,шт,NaN
id,,,,,,,,
245,008-1,858.0,ultra концентрат 10 конструкция 1 антисептик н...,1.0,0.0,1.0,0.0,0.0
3,242-12,1075.0,состав антигололед готовый 32,12.0,1.0,0.0,0.0,0.0
443,0024-06 с,644.0,п цвет герметика шов деревянный конструкция ак...,0.6,0.0,1.0,0.0,0.0
147,305-2,342.0,crystal кондиционер королевский с концентрат а...,2.0,0.0,1.0,0.0,0.0
502,0024-7 б,891.0,цвет герметик белый акриловой,7.0,1.0,0.0,0.0,0.0


In [ ]:
train = dealer_price_final.merge(product_match, how='left',
                                 left_on='product_key',
                                 right_on='key')[['price', 'name_new', 'article',
                                                  'quantity', 'кг', 'л', 'шт',
                                                  np.nan, 'product_id']]


In [ ]:
nltk.download('stopwords')

stopwords=set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))

count_tf_idf.fit(pd.concat([base['full_name'], train['name_new']]))

feature_base_tfidf = count_tf_idf.transform(base['full_name'])
feature_train_tfidf = count_tf_idf.transform(train['name_new'])

base_tfidf = pd.DataFrame(feature_base_tfidf.toarray()).set_index(base.index)
base_tfidf = pd.concat([base_tfidf, base[['article', 'recommended_price',
                                          'quantity', 'кг', 'л', 'шт', np.nan]]], axis=1)

train_tfidf = pd.DataFrame(feature_train_tfidf.toarray())
train_tfidf['product_id'] = train['product_id']
train_tfidf = pd.concat([train_tfidf, train[['price', 'article', 'quantity',
                                             'кг', 'л', 'шт', np.nan]]], axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
base_tfidf.columns = base_tfidf.columns.astype('str')
train_tfidf.columns = train_tfidf.columns.astype('str')

base_tfidf = base_tfidf.drop(['recommended_price'], axis=1)
train_tfidf = train_tfidf.drop(['price'], axis=1)

In [ ]:
base_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,817,818,819,820,article,quantity,кг,л,шт,nan
id,,,,,,,,,,,,,,,,,,,,,
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,008-1,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,242-12,12.0,1.0,0.0,0.0,0.0
443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0024-06 с,0.6,0.0,1.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,305-2,2.0,0.0,1.0,0.0,0.0
502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0024-7 б,7.0,1.0,0.0,0.0,0.0


In [ ]:
num = list(base_tfidf.columns)
num.remove('article')

In [ ]:
scaler = MinMaxScaler()
base_tfidf_scaled = pd.DataFrame(scaler.fit_transform(base_tfidf[num]), columns=scaler.feature_names_in_).set_index(base_tfidf.index)

In [ ]:
targets = train_tfidf["product_id"]
train_tfidf.drop("product_id", axis=1, inplace=True)

num1 = list(train_tfidf.columns)
num1.remove('article')

In [ ]:
train_tfidf_scaled = pd.DataFrame(scaler.transform(train_tfidf[num1]), columns=scaler.feature_names_in_)

In [ ]:
base_tfidf_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,quantity,кг,л,шт,nan
id,,,,,,,,,,,,,,,,,,,,,
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.184615,1.0,0.0,0.0,0.0
443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.009231,0.0,1.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.030769,0.0,1.0,0.0,0.0
502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.107692,1.0,0.0,0.0,0.0


In [ ]:
train_tfidf_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,quantity,кг,л,шт,nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.007692,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.007692,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.007692,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,1.0,0.0,0.0


## Перевод в классификацию

In [ ]:
def accuracy_aggregator(targets, idx, base_index):
  acc = 0
  for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el[:25]])
  result = 100 * acc / len(idx)
  return result

In [ ]:
dims = base_tfidf_scaled.shape[1]
n_cells = 1 # количество центроидов
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

# подготовка к поиску
idx_l2.train(np.ascontiguousarray(base_tfidf_scaled.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(base_tfidf_scaled.values).astype('float32'))

# создание словаря для нахождения индекса товара в базовом наборе данных
base_index = {k: v for k, v in enumerate(base_tfidf_scaled.index.to_list())}
vecs, idx = idx_l2.search(np.ascontiguousarray(train_tfidf_scaled.values).astype('float32'), 25)

In [ ]:
vecs.shape

(20416, 25)

In [ ]:
accuracy_aggregator(targets, idx, base_index)

82.3912617554859

In [ ]:
df_for_cb = pd.DataFrame()

# для каждого запроса из обучающего набора данных
for i in range(train_tfidf.shape[0]):

    # берем индексы FAISS для первых пяти рекомендаций

    df = pd.DataFrame(idx[i])
    # берем расстояния от запроса до рекомендаций FAISS
    df = pd.concat([df, pd.DataFrame(vecs[i])], axis=1)
    df.columns = ['idx', 'distance']
    # индекс вектора-запроса для получения его координат
    df['query_idx'] = i
    # индекс вектора рекомендаций экспертов
    df['target_idx'] = targets[i]
    df_for_cb = pd.concat([df_for_cb, df], ignore_index=True)
# функция для получения индекса рекомендаций в базовом наборе товаров по индексу FAISS
def get_base_idx(row):
    return base_index[row['idx']]
# по внутреннему индексу FAISS восстанавливаем индекс рекомендованного товара в базовом наборе
df_for_cb['base_idx'] = df_for_cb.apply(get_base_idx, axis=1)

df_for_cb['target'] = df_for_cb['target_idx'] == df_for_cb['base_idx']
df_for_cb['target'] = df_for_cb['target'].astype('int')

# добавляем координаты векторов-запросов
#df_for_cb = df_for_cb.merge(train_tfidf, how='inner', left_on='query_idx', right_index=True)

# добавляем координаты векторов-правильных ответов FAISS
#df_for_cb = df_for_cb.merge(base_tfidf, how='inner', left_on='base_idx', right_index=True)

In [ ]:
df_for_cb = df_for_cb.dropna(subset=['target_idx'])

In [ ]:
df_for_cb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439175 entries, 0 to 510349
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   idx         439175 non-null  int64  
 1   distance    439175 non-null  float32
 2   query_idx   439175 non-null  int64  
 3   target_idx  439175 non-null  float64
 4   base_idx    439175 non-null  int64  
 5   target      439175 non-null  int64  
dtypes: float32(1), float64(1), int64(4)
memory usage: 21.8 MB


In [ ]:
df_for_cb['query_idx'].unique()

array([    0,     1,     2, ..., 20411, 20412, 20413])

In [ ]:
df_for_cb.shape

(439175, 6)

In [ ]:
total_table = pd.DataFrame()
total_table = df_for_cb.merge(train, how='left',
                              left_on='query_idx',
                              right_index=True).\
                        merge(base, how='left',
                              left_on='base_idx',
                              right_index=True)
total_table.head()

,idx,distance,query_idx,target_idx,base_idx,target,price,name_new,article_x,quantity_x,...,nan_x,product_id,article_y,recommended_price,full_name,quantity_y,кг_y,л_y,шт_y,nan_y
0,178,5.410485,0,12.0,15,0,233.0,средство универсальный universal spray,NaN,0.5,...,0.0,12.0,106-5,1538.0,мыть средство чистить spray 100 и концентрат у...,5.0,0.0,1.0,0.0,0.0
1,488,6.782314,0,12.0,12,1,233.0,средство универсальный universal spray,NaN,0.5,...,0.0,12.0,105-00,222.0,мыть средство чистить spray готовый состав и у...,0.5,0.0,1.0,0.0,0.0
2,343,7.153287,0,12.0,13,0,233.0,средство универсальный universal spray,NaN,0.5,...,0.0,12.0,105-5,853.0,мыть профессиональный средство чистить spray г...,5.0,0.0,1.0,0.0,0.0
3,435,8.227001,0,12.0,4,0,233.0,средство универсальный universal spray,NaN,0.5,...,0.0,12.0,104-1,352.0,мыть профессиональный средство 200 prof 10 кон...,1.0,0.0,1.0,0.0,0.0
4,257,8.387577,0,12.0,259,0,233.0,средство универсальный universal spray,NaN,0.5,...,0.0,12.0,005-1,748.0,для universal универсальный universalконцентра...,1.0,0.0,1.0,0.0,0.0


In [ ]:
total_table = total_table.reset_index(drop=True)

In [ ]:
total_table.shape

(439175, 23)

In [ ]:
# запоминание id правильных ответов
subd = {k: v for k, v in enumerate(total_table['base_idx'].to_list())}

In [ ]:
total_table = total_table.drop(['idx', 'query_idx',
         'target_idx', 'base_idx', 'name_new',
         'product_id', 'full_name'], axis=1)

In [ ]:
total_table.head()

,distance,target,price,article_x,quantity_x,кг_x,л_x,шт_x,nan_x,article_y,recommended_price,quantity_y,кг_y,л_y,шт_y,nan_y,article
0,5.410485,0,233.0,NaN,0.5,0.0,1.0,0.0,0.0,106-5,1538.0,5.0,0.0,1.0,0.0,0.0,unknown
1,6.782314,1,233.0,NaN,0.5,0.0,1.0,0.0,0.0,105-00,222.0,0.5,0.0,1.0,0.0,0.0,unknown
2,7.153287,0,233.0,NaN,0.5,0.0,1.0,0.0,0.0,105-5,853.0,5.0,0.0,1.0,0.0,0.0,unknown
3,8.227001,0,233.0,NaN,0.5,0.0,1.0,0.0,0.0,104-1,352.0,1.0,0.0,1.0,0.0,0.0,unknown
4,8.387577,0,233.0,NaN,0.5,0.0,1.0,0.0,0.0,005-1,748.0,1.0,0.0,1.0,0.0,0.0,unknown


In [ ]:
total_table['article'] = total_table['article_x'].isna()
for i in range(total_table.shape[0]):
  if total_table['article'][i] == True:
    total_table['article'][i]='unknown'
  else:
    total_table['article'][i] = (total_table['article_x'][i] == total_table['article_y'][i]) * 1

In [ ]:
total_table['quantity'] = (total_table['quantity_x'] == total_table['quantity_y']) * 1
total_table['kg'] = (total_table['кг_x'] == total_table['кг_y']) * 1
total_table['l'] = (total_table['л_x'] == total_table['л_y']) * 1
total_table['cnt'] = (total_table['шт_x'] == total_table['шт_y']) * 1
total_table['nan'] = (total_table['nan_x'] == total_table['nan_y']) * 1
total_table['diff_price'] = np.abs(total_table['price'] - total_table['recommended_price'])

In [ ]:
total_table['article'] = total_table['article'].astype('str')

In [ ]:
total_table = total_table.drop(['article_x', 'article_y', 'quantity_x',
              'quantity_y', 'кг_x', 'кг_y', 'л_x',
              'л_y', 'шт_x', 'шт_y', 'nan_x', 'nan_y',
              'recommended_price', 'price'], axis=1)

In [ ]:
total_table.head()

,distance,target,article,quantity,kg,l,cnt,nan,diff_price
0,5.410485,0,unknown,0,1,1,1,1,1305.0
1,6.782314,1,unknown,1,1,1,1,1,11.0
2,7.153287,0,unknown,0,1,1,1,1,620.0
3,8.227001,0,unknown,0,1,1,1,1,119.0
4,8.387577,0,unknown,0,1,1,1,1,515.0


In [ ]:
total_scaler = MinMaxScaler()
total_table[['distance', 'diff_price']] = total_scaler.fit_transform(total_table[['distance', 'diff_price']])

In [ ]:
total_encoder = OneHotEncoder()

dem = total_encoder.fit_transform(total_table[['article']]).toarray()
total_table = pd.concat([total_table, pd.DataFrame(data=dem, columns=total_encoder.categories_[0])], axis=1).drop(['article'], axis=1)

In [ ]:
total_table = total_table.rename({"0":"art_0","1":"art_1", "unknown":"art_unknown"}, axis='columns')

In [ ]:
total_table.head()

,distance,target,quantity,kg,l,cnt,nan,diff_price,art_0,art_1,art_unknown
0,0.272812,0,0,1,1,1,1,0.089206,0.0,0.0,1.0
1,0.341984,1,1,1,1,1,1,0.000752,0.0,0.0,1.0
2,0.360689,0,0,1,1,1,1,0.042382,0.0,0.0,1.0
3,0.414829,0,0,1,1,1,1,0.008135,0.0,0.0,1.0
4,0.422926,0,0,1,1,1,1,0.035204,0.0,0.0,1.0


In [ ]:
total_table.to_csv('for_model_last.csv')

In [ ]:
features_train = total_table.drop(['target'], axis=1)
target_train = total_table['target']

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
                                    [features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat(
                                    [target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
                                    features_downsampled, target_downsampled, random_state=42)
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.05)

In [ ]:
target_downsampled.value_counts()

0    21118
1    16821
Name: target, dtype: int64

In [ ]:
target_train, target_test = train_test_split(target_downsampled, random_state=1, test_size=.25)
features_train, features_test = train_test_split(features_downsampled, random_state=1, test_size=.25)
print(features_train.shape, features_test.shape)

(28454, 10) (9485, 10)


In [ ]:
model = DecisionTreeClassifier(class_weight='balanced')
model.fit(features_train, target_train)
pred = model.predict(features_test)
accuracy_score(target_test, pred)

0.9816552451238798

In [ ]:
model.feature_importances_

array([2.19295042e-01, 5.32879550e-01, 3.89119931e-03, 2.46362133e-02,
       1.42283411e-04, 3.52558675e-04, 1.58289582e-01, 3.43687236e-02,
       2.07844832e-02, 5.36036403e-03])

In [ ]:
pd.DataFrame(data=model.feature_importances_, index=features_test.columns)

,0
distance,0.219295
quantity,0.532880
kg,0.003891
l,0.024636
cnt,0.000142
nan,0.000353
diff_price,0.158290
art_0,0.034369
art_1,0.020784
art_unknown,0.005360


In [ ]:
model.predict_proba

<bound method LogisticRegression.predict_proba of LogisticRegression(class_weight='balanced')>

In [ ]:
pred